# 🚀 Meta-Refine on Kaggle

## Running Meta Llama 3.1-8B for Claude Code-Level Analysis

**Kaggle advantages:**
- ✅ 16GB RAM (vs 12GB Colab)
- ✅ 30 hours/week free GPU
- ✅ Better for ML workloads

**Enable GPU:** Settings → Accelerator → GPU T4 x2

In [ ]:
# Check system resources
!nvidia-smi
!free -h
print("GPU Memory and System RAM available for Meta Llama 8B")

In [ ]:
# Clone repo and install
!git clone https://github.com/abritton2002/meta-refine.git
import os
os.chdir('/kaggle/working/meta-refine')

!pip install -r requirements.txt
!pip install accelerate bitsandbytes transformers torch --upgrade

In [ ]:
# Configure for Kaggle GPU environment
import os

# IMPORTANT: Replace with your actual HF token
HF_TOKEN = "hf_YOUR_ACTUAL_TOKEN_HERE"  # ⚠️ Replace this!

# Kaggle-optimized settings
os.environ['HF_TOKEN'] = HF_TOKEN
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

config = f"""
HF_TOKEN={HF_TOKEN}
MODEL_NAME=meta-llama/Llama-3.1-8B-Instruct
MODEL_DEVICE=cuda
MODEL_TEMPERATURE=0.3
MODEL_LOAD_IN_4BIT=true
TORCH_DTYPE=float16
MAX_FILE_SIZE=1000000
CHUNK_SIZE=2000
"""

with open('.env', 'w') as f:
    f.write(config)

print("✅ Kaggle environment configured for Meta Llama 8B")
print("🔧 4-bit quantization enabled for 16GB RAM")
print("⚠️  Remember to replace HF_TOKEN with your actual token!")

In [ ]:
# Test Meta-Refine with Meta Llama
print("🧠 Loading Meta Llama 3.1-8B-Instruct...")
print("⏳ First run: Model download + load (~10-15 minutes)")
print("💾 Using 4-bit quantization for Kaggle's 16GB RAM")

# Run analysis on test file
!python -m meta_refine_pkg.cli analyze --file examples/test_code.py --verbose

In [ ]:
# Project-wide analysis
print("🏗️ Analyzing entire Meta-Refine codebase...")
!python -m meta_refine_pkg.cli analyze --project meta_refine_pkg --format json --output kaggle_results.json

In [ ]:
# View and download results
import json
import os

if os.path.exists('kaggle_results.json'):
    print("📊 Analysis Results Summary:")
    
    with open('kaggle_results.json', 'r') as f:
        results = json.load(f)
    
    # Display summary
    summary = results['meta_refine_analysis']['summary']
    print(f"Files analyzed: {summary['total_files']}")
    print(f"Issues found: {summary['total_issues']}")
    print(f"Analysis time: {summary['total_analysis_time']:.2f}s")
    
    # Show severity breakdown
    severity = summary.get('severity_breakdown', {})
    print(f"\n🚨 Issues by severity:")
    for sev, count in severity.items():
        print(f"  {sev}: {count}")
    
    # Show sample issues
    print(f"\n📋 Sample Issues Found:")
    for file_result in results['meta_refine_analysis']['files'][:3]:
        if file_result['issues']:
            print(f"\n📁 {file_result['file_path']}:")
            for issue in file_result['issues'][:2]:
                print(f"  • {issue['severity']}: {issue['description'][:80]}...")
                
else:
    print("❌ No results file found - check if analysis completed successfully")
    print("📋 Available files:")
    !ls -la *.json 2>/dev/null || echo "No JSON files found"

In [ ]:
# Test custom code analysis
print("🔧 Test with your own code:")
print("Copy-paste your code below and run analysis")

# Example: Analyze custom code
custom_code = """
def risky_function():
    password = "secret123"  # Hardcoded secret
    user_input = input("Enter SQL: ")
    query = f"SELECT * FROM users WHERE id = {user_input}"  # SQL injection
    return eval(user_input)  # Code injection
"""

# Save and analyze
with open('custom_test.py', 'w') as f:
    f.write(custom_code)

print("\n🔍 Analyzing custom code with Meta Llama...")
!python -m meta_refine_pkg.cli analyze --file custom_test.py